In [ ]:
import davidstools as my
import pandas as pd
import matplotlib.pyplot as plt
from seaborn import regplot
from sklearn.linear_model import LinearRegression
from sklearn.metrics import root_mean_squared_error
import numpy as np

COVID_DATA = pd.read_csv('./data/super_covid_data.csv')
DEATH_DATA = pd.read_csv('./data/covid_deaths_usafacts.csv')
CASE_DATA = pd.read_csv('./data/covid_confirmed_usafacts.csv')
POPULATION = pd.read_csv('./data/covid_county_population_usafacts.csv')
META_COLUMNS = ['countyFIPS', 'County Name', 'State', 'StateFIPS']
EMPLOYMENT = pd.read_excel('./data/allhlcn20.xlsx')

data = [CASE_DATA, DEATH_DATA]
texas = []

for frame in data:
    subframe = my.extract_state_data('TX', frame)
    texas.append(my.select_dates(subframe, '2020-06-01', '2021-01-03'))

In [ ]:
texas_5 = []

for frame in texas:
    subframe = my.top_5(frame, '2021-01-03')
    texas_5.append(subframe)

texas_5_change = []

for frame in texas_5:
    subframe = my.correct_numbers(frame)
    texas_5_change.append(subframe)


titles = ['New Cases Per 10000 people', 'New Deaths Per 100000 people']
norm = [10000, 100000]
variables = ['Cases', 'Deaths']
i = 0
for frame in texas_5_change:
    normFrame = my.normalize_pop(frame, POPULATION, norm[i])
    transpose = normFrame.drop(columns=['countyFIPS','State', 'StateFIPS']).set_index('County Name').T  
    transpose.plot(kind='line')
    plt.title(titles[i])
    plt.xlabel('Date')
    plt.ylabel(variables[i])
    i+=1

    

In [ ]:
i=0
for frame in texas_5_change:
    new_frame = frame.drop(columns=['countyFIPS','State', 'StateFIPS']).set_index('County Name')
    counties = new_frame.index.to_list()
    x = new_frame.T.index.to_list()
    for county in counties:
        plt.scatter(y=new_frame.loc[county], x=x)
    plt.legend(counties)
    plt.title(titles[i])
    plt.xlabel('Days')
    plt.ylabel(variables[i])
    plt.show()
    i+=1



In [ ]:
i=0
for frame in texas_5_change:
    normFrame = my.normalize_pop(frame, POPULATION, norm[i])
    new_frame = normFrame.drop(columns=['countyFIPS','State', 'StateFIPS']).set_index('County Name')
    counties = new_frame.index.to_list()
    x = new_frame.T.index.to_list() 

    X = np.arange(len(new_frame.T)).reshape(-1, 1)

    for county in counties:
        plt.scatter(y=new_frame.loc[county], x=x, alpha=0.2)

    j=0
    for county in counties:
        y_true = new_frame.loc[county]
        y = new_frame[new_frame.index == county].T
        regr = LinearRegression()
        regr.fit(X, y)
        y_pred = regr.predict(X)

        error = root_mean_squared_error(y_true, y_pred, multioutput='raw_values')
        message = f"The RMSE of {counties[j]} is {error}"

        print(message)
        j+=1

        plt.plot(x, y_pred, linewidth=3)
    plt.legend(counties)
    plt.title(titles[i])
    plt.xlabel('Days')
    plt.ylabel(variables[i])
    plt.show()
    i+=1

In [ ]:
i=0
for frame in texas_5_change:
    normFrame = my.normalize_pop(frame, POPULATION, norm[i])
    new_frame = normFrame.drop(columns=['countyFIPS','State', 'StateFIPS']).set_index('County Name')
    counties = new_frame.index.to_list()
    x = new_frame.T.index.to_list()

    x_array = np.arange(len(x))

    X = np.arange(len(new_frame.T)).reshape(-1, 1)

    colors = ['tab:blue', 'tab:orange', 'tab:green', 'tab:red', 'tab:purple']

    j=0
    for county in counties:
        y = new_frame[new_frame.index == county].T

        regplot(x=x_array, y=y, color=colors[j], marker='x')

        plt.legend(counties)
        plt.title(titles[i])
        plt.xlabel('Days')
        plt.ylabel(variables[i])
        plt.show()
        j+=1
    i+=1

In [ ]:
i=0
for frame in texas_5_change:
    normFrame = my.normalize_pop(frame, POPULATION, norm[i])
    new_frame = normFrame.drop(columns=['countyFIPS','State', 'StateFIPS']).set_index('County Name')
    counties = new_frame.index.to_list()
    x = new_frame.T.index.to_list()

    x_array = np.arange(len(x))

    X = np.arange(len(new_frame.T)).reshape(-1, 1)

    colors = ['tab:blue', 'tab:orange', 'tab:green', 'tab:red', 'tab:purple']

    j=0
    for county in counties:
        y = new_frame[new_frame.index == county].T

        regplot(x=x_array, y=y, color=colors[j], order=3, marker='x')

        plt.legend(counties)
        plt.title(titles[i])
        plt.xlabel('Days')
        plt.ylabel(variables[i])
        plt.show()
        j+=1
    i+=1

# Hypothesis Testing

## Predicitons:

I think there will be a negative correlation between annual average establishment count in the health and education sector and deaths.

I think there will be a positive correlation between anual average establishment count in liesure and hospitality and cases.

I think there will be a positive correlation between anual average employment in service providing industries and cases.

In [64]:
# Construct a Dataframe holding the data I am curious about.

mergeable_employment = EMPLOYMENT.copy()
mergeable_employment = mergeable_employment[mergeable_employment['St Name'] == 'Texas']
texas_employment = mergeable_employment[mergeable_employment['Area Type'] == 'County']

#texas_employment['countyFIPS'] = 
texas_employment = texas_employment.drop(columns=['Area\nCode'])
start = mergeable_employment['Area\nCode'].astype(int)
texas_employment.insert(0, 'countyFIPS',start)
# Health and Education data
texas_HE = texas_employment[texas_employment['Industry'] =='1025 Education and health services']
# Liesure and Hospitality data
texas_LH = texas_employment[texas_employment['Industry'] =='1026 Leisure and hospitality']
#Service providing
texas_SP = texas_employment[texas_employment['Industry'] =='102 Service-providing']


texas_deaths = my.select_dates(my.extract_state_data('TX', DEATH_DATA), '2021-01-03', '2021-01-03')
texas_cases = my.select_dates(my.extract_state_data('TX', CASE_DATA), '2021-01-03', '2021-01-03')

texas_HE_C = texas_cases.merge(texas_HE, how='inner', on='countyFIPS')
texas_LH_C= texas_cases.merge(texas_LH, how='inner', on='countyFIPS')
texas_SP_C= texas_cases.merge(texas_SP, how='inner', on='countyFIPS')

texas_HE_D = texas_deaths.merge(texas_HE, how='inner', on='countyFIPS')
texas_LH_D= texas_deaths.merge(texas_LH, how='inner', on='countyFIPS')
texas_SP_D= texas_deaths.merge(texas_SP, how='inner', on='countyFIPS')

texas_HE_D

,countyFIPS,County Name,State,StateFIPS,2021-01-03,St,Cnty,Own,NAICS,Year,...,Ownership,Industry,Annual Average Status Code,Annual Average Establishment Count,Annual Average Employment,Annual Total Wages,Annual Average Weekly Wage,Annual Average Pay,Employment Location Quotient Relative to U.S.,Total Wage Location Quotient Relative to U.S.
0,48001,Anderson County,TX,48,75,48,1.0,5,1025,2020,...,Private,1025 Education and health services,NaN,220,2194,92362191,809,42093,0.70,0.70
1,48003,Andrews County,TX,48,33,48,3.0,5,1025,2020,...,Private,1025 Education and health services,NaN,24,160,5707996,685,35619,0.14,0.09
2,48005,Angelina County,TX,48,176,48,5.0,5,1025,2020,...,Private,1025 Education and health services,NaN,284,7521,322909934,826,42934,1.39,1.59
3,48007,Aransas County,TX,48,25,48,7.0,5,1025,2020,...,Private,1025 Education and health services,NaN,54,450,19038322,813,42292,0.52,0.63
4,48009,Archer County,TX,48,6,48,9.0,5,1025,2020,...,Private,1025 Education and health services,NaN,19,95,2367784,478,24880,0.37,0.27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
248,48499,Wood County,TX,48,85,48,499.0,5,1025,2020,...,Private,1025 Education and health services,NaN,117,1669,57485783,662,34447,1.01,0.99
249,48501,Yoakum County,TX,48,21,48,501.0,5,1025,2020,...,Private,1025 Education and health services,NaN,10,111,3501309,605,31472,0.23,0.13
250,48503,Young County,TX,48,32,48,503.0,5,1025,2020,...,Private,1025 Education and health services,NaN,57,668,25498750,735,38200,0.65,0.64
251,48505,Zapata County,TX,48,23,48,505.0,5,1025,2020,...,Private,1025 Education and health services,NaN,22,350,8006114,440,22869,0.76,0.42
